# Leslibraires

___

In [37]:
import pandas as pd
import os
import glob
import mysql.connector

In [38]:
os.chdir("C:\Adrian - GDrive\Professionnel\Formation\Informatique - Digital\Data University\Projet certification\src\leslibraires")

df = pd.DataFrame(columns=[], index=[])

for file in glob.glob("*.xlsx"):
    #print(file)
    csv = pd.read_excel(file, sheet_name="Lignes de commandes particulier")
    print(csv.columns)
    df = df.append(csv, ignore_index=True)

df.shape

Index(['Commande', 'Expédition', 'Date de commande', 'Date d'expédition',
       'ean13', 'Titre', 'Auteur', 'Éditeur', 'Quantité', 'Total HT',
       'Total TTC'],
      dtype='object')
Index(['Commande', 'Expédition', 'Date de commande', 'Date d'expédition',
       'ean13', 'Titre', 'Auteur', 'Éditeur', 'Quantité', 'Total HT',
       'Total TTC'],
      dtype='object')
Index(['Commande', 'Expédition', 'Date de commande', 'Date d'expédition',
       'ean13', 'Titre', 'Auteur', 'Éditeur', 'Quantité', 'Total HT',
       'Total TTC'],
      dtype='object')
Index(['Commande', 'Expédition', 'Date de commande', 'Date d'expédition',
       'ean13', 'Titre', 'Auteur', 'Éditeur', 'Quantité', 'Total HT',
       'Total TTC'],
      dtype='object')
Index(['Commande', 'Expédition', 'Date de commande', 'Date d'expédition',
       'ean13', 'Titre', 'Auteur', 'Éditeur', 'Quantité', 'Total HT',
       'Total TTC'],
      dtype='object')
Index(['Commande', 'Expédition', 'Date de commande', 'Date d'expé

(542, 11)

In [39]:
df.columns

Index(['Commande', 'Expédition', 'Date de commande', 'Date d'expédition',
       'ean13', 'Titre', 'Auteur', 'Éditeur', 'Quantité', 'Total HT',
       'Total TTC'],
      dtype='object')

In [40]:
df[df["ean13"].notna()].shape

(542, 11)

In [41]:
df = df.drop(columns=['Commande', 'Expédition', "Date d'expédition", 'Titre', 'Auteur', 'Éditeur', 'Quantité', 'Total HT'])


In [42]:
df.shape

(542, 3)

In [43]:
df.dtypes

Date de commande    datetime64[ns]
ean13                        int64
Total TTC                  float64
dtype: object

In [44]:
df.head()

,Date de commande,ean13,Total TTC
0,2018-12-31 14:01:25,9782266070188,16.95
1,2018-12-31 10:10:33,9782020055048,19.80
2,2019-01-01 19:15:28,2000116421223,19.80
3,2019-01-02 12:02:21,2000053168526,24.90
4,2019-01-03 07:05:27,9782262007997,29.80


In [45]:
df["Date de commande"] = df["Date de commande"].dt.strftime("%Y-%m-%d") 
df.head()

,Date de commande,ean13,Total TTC
0,2018-12-31,9782266070188,16.95
1,2018-12-31,9782020055048,19.80
2,2019-01-01,2000116421223,19.80
3,2019-01-02,2000053168526,24.90
4,2019-01-03,9782262007997,29.80


## Récupération du SKU

In [46]:
try:
    connection = mysql.connector.connect(host='localhost',
                                         database='db_lelivre_pro',
                                         user='localscript',
                                         password='GY9x9xLyaJA8CaqQMG6S',
                                         port='3308')
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if (connection.is_connected()):
        ### Dataframe 1
        var_sql = "SELECT ref as sku, ean as ean13 FROM db_lelivre_pro.livresv2 where date > 20181231 and date < 20190901"
        df_sku = pd.read_sql(var_sql, connection)
        
        print("Dataframe constitué")
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

Connected to MySQL Server version  8.0.18
You're connected to database:  ('db_lelivre_pro',)
Dataframe constitué
MySQL connection is closed


In [47]:
df_sku.shape

(78921, 2)

In [48]:
df.shape

(542, 3)

In [49]:
df_sku.dtypes

sku      object
ean13    object
dtype: object

In [50]:
df.dtypes

Date de commande     object
ean13                 int64
Total TTC           float64
dtype: object

In [51]:
df["ean13"]= df["ean13"].astype(str)

In [52]:
df = df.merge(df_sku, on="ean13")

In [53]:
df = df.drop(columns="ean13")

In [54]:
df

,Date de commande,Total TTC,sku
0,2018-12-31,16.95,ROD0138137
1,2018-12-31,19.80,R160184541
2,2019-01-03,29.80,R320095702
3,2019-01-02,30.90,R200074925
4,2019-01-04,24.90,RO60137276
...,...,...,...
287,2019-08-27,65.25,R200032571
288,2019-08-28,35.80,R200020378
289,2019-08-26,39.80,R260218629
290,2019-08-29,49.50,R160200381


In [55]:
os.chdir("C:\Adrian - GDrive\Professionnel\Formation\Informatique - Digital\Data University\Projet certification\csv")

df.to_csv("1-leslibraires.csv", index=False)